In [ ]:
# !pip install llama_index==0.11.4
# !pip install PyYAML
# !pip install plotly==5.24.0
# !pip install docx2txt==0.8
# !pip install chromadb==0.5.5
# !pip install llama-index-vector-stores-chroma==0.2.0
# !pip install llama-index-extractors-entity==0.2.0
# !pip install llama-index-readers-web==0.2.2
# !pip install transformers==4.40.2

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
import openai

openai.api_key = "sk-proj-your-openai-api-key"
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.2)

# Retriever


In [71]:
from llama_index.core import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import (
    SummaryIndex, SimpleDirectoryReader, get_response_synthesizer
)

text = """Mèo rất đáng yêu.
        Acid hydrochloric đậm đặc nhất có nồng độ tối đa là 40%.
        Ở dạng đậm đặc, acid này có thể tạo thành các sương mù acid,
        chúng đều có khả năng ăn mòn các mô con người, gây tổn thương cơ quan hô hấp,
        mắt, da và ruột."""

doc = Document(text=text)

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=50, chunk_overlap=0),
        TitleExtractor(),
        OpenAIEmbedding(),
    ]
)

# run the pipeline
nodes = pipeline.run(documents=[doc])

# Create index from nodes
index = VectorStoreIndex(nodes)

100%|██████████| 4/4 [00:02<00:00,  1.66it/s]


In [75]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    verbose=True
)

pp = SimilarityPostprocessor(similarity_cutoff=0.5)

In [76]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[pp]
)

In [77]:
response = query_engine.query(
    "Mèo rất đáng yêu phải không?"
)
print(response)

1 text chunks after repacking
Đúng vậy, mèo rất đáng yêu.
